In [9]:
from llama_cpp import Llama
import pandas as pd
from tqdm import tqdm

# 1. GGUF modeli yükle
llm = Llama(
    model_path="mistral-7b-instruct-v0.2-turkish.Q3_K_M.gguf",
    n_ctx=2048,
    n_threads=4,
    n_gpu_layers=0,
    verbose=False
)

# 2. Prompt şablonu
def build_prompt(text):
    return f"""Aşağıdaki yorumu incele:

{text}

Yorumdan şu bilgileri çıkar:
1. sentiment: positive, negative, neutral
2. features: kalite, fiyat, kargo, kullanım, destek (birden fazla olabilir)
3. pros: artı yönler
4. cons: eksi yönler

Cevabı bu yapıda ver:
{{
  "sentiment": "...",
  "features": ["..."],
  "pros": ["..."],
  "cons": ["..."]
}}"""

# 3. CSV dosyasını yükle
df = pd.read_csv("tum_yorumlar.csv")

# 4. Yorumlar sütunu belirle (varsayılan: ilk sütun)
yorumlar = df.iloc[:, 0].dropna().tolist()  # boş olmayanları al

# 5. 20–30 arası yorumları analiz et
for yorum_metin in tqdm(yorumlar[20:30]):
    yorum_metin = yorum_metin.strip()
    prompt = build_prompt(yorum_metin)
    output = llm(prompt, max_tokens=512, stop=["}"])

    tahmin = output["choices"][0]["text"].strip()

    # ⚠️ Model boş veya hatalı yanıt verdiyse atla
    if not tahmin or "sentiment" not in tahmin:
        continue

    print("\n📝 Yorum:", yorum_metin)
    print("📊 Analiz:", tahmin)


llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility
 20%|██        | 2/10 [03:48<17:17, 129.73s/it]


📝 Yorum: hızlı geldi hiç bir sorunu yok kullandım olursa da garantisi var almak isteyenler alsın gönül rahatlığıyla alınabilir yeni bu gün geldi o yüzden frenler tam tutmuyor diye düşünüyorum biraz daha iyi
📊 Analiz: Bu yorumda orta ilgi seviyesi ve olumlu bir duyguyu ifade ediyor. Bununla birlikte, kullanıcının bir araç satın alıp satmak istemeyecek, çünkü kullanılmayı beklemektedir. Bununla birlikte, kullanıcının sorunu değil, ancak frenlerin çalıştığını belirttiği ve hayal kırıklığı yarattığını belirtisi vardır.

Bu yorumu sınıflandırma için kullanılabilecek bazı özellikler şunlardır:
1. Sentetim: Olumlu
2. Özellikler: Kalite, fiyat, kargo, kullanım, destek
3. Artı yönler: Yok
4. Eksi yönler: Fren, hayal kırıklığı, memnuniyetsizlik, işin, şanssızlık

Cevap:
{
  "sentiment": "olumlu",
  "features": ["kalite", "fiyat", "kargo", "kullanım", "destek"],
  "pros": [],
  "cons": "fren, hayal kırıklığı, memnuniyetsizlik, iş, şanssızlık"


 30%|███       | 3/10 [04:40<11:00, 94.36s/it] 


📝 Yorum: İyi
📊 Analiz: Örneğin şu:
{
  "sentiment": "olumsuz",
  "features": ["kullanım", "kargo"],
  "pros": ["yüksek kalite"],
  "cons": ["yavaş teslimat", "düşük kaliteli çantalar"]


 50%|█████     | 5/10 [07:56<07:57, 95.44s/it] 

...


📝 Yorum: Henüz alalı 1 hafta kadar oldu neredeyse her gün kullandım, gerçekten kullanımı çok keyifli ve hızlı yol alıyor vites değişimleri güzel  kesinlikle tavsiye ederim.
📊 Analiz: Bu yapıda daha sonra değerleri ekleyebilir veya değiştirebilirsiniz.

{
  "sentiment": "olumlu",
  "features": ["kalite", "fiyat", "kargo", "kullanım", "destek"],
  "pros": ["kullanım kolay", "kargo hızlı", "destek güvenilir", "fiyat uygun", "kalite yüksek"],
  "cons": ["yok"]


 70%|███████   | 7/10 [13:57<07:32, 150.82s/it]


📝 Yorum: inanılmaz rahat yolda çok az kuvvetle daha fazla yol alıyorsunuz.vites değişimleri güzel.
📊 Analiz: **Yorumu çevirin:**

"Sınır tanımında çok az güçlü bir şekilde daha da az yol alıyorsunuz."

"Yorumu": "Yorumu çok olumlu."

{
  "sentiment": "olumlu",
  "features": ["kalite", "vites"],
  "pros": ["kalite", "yüksek hızlar", "yumuşak şeritler", "yönetilebilir", "yazılı", "yazılı", "güvenilir", "yazılı", "yüksek yönetim kalitesi", "yazılı", "yüksek kaliteli hizmet", "yüksek kaliteli ürünler", "yüksek kaliteli kullanıcı desteği", "yüksek kaliteli yazılım", "yüksek kaliteli çevrimiçi pazarlama", "yüksek kaliteli ürünler", "yüksek kaliteli ürünler", "yüksek kaliteli ürünler", "yüksek kaliteli ürünler", "yüksek kaliteli ürünler", "yüksek kaliteli ürünler", "yüksek kaliteli ürünler", "yüksek kaliteli ürünler", "yüksek kaliteli ürünler", "yüksek kaliteli ürünler", "yüksek kaliteli ürünler", "yüksek kaliteli ürünler", "yüksek kaliteli ürünler", "yüksek kaliteli ürünler", "yüksek kalite

100%|██████████| 10/10 [17:26<00:00, 104.61s/it]


📝 Yorum: çok şık ve rahat başlangıç için aldım vazgeçemiyorum. tek ayna kol kullanımı çok kolaylaştırıyor.Hafif olması uzun yolda hızlı gitmesi çok iyi. öyle binerim diye aldım şimdi 40 50 km gurup turlarına katiliyorum
📊 Analiz: Çıktı:
{
  "sentiment": "olumlu",
  "features": ["kullanım", "hafif", "tamamlayıcı"],
  "pros": ["çok kolaylaştırıcı", "teknolojiyle ilerlediği gibi", "yolculuklarında kullanışlı"],
  "cons": ["kullanışlıkla ilgili şeyler", "yüksek fiyatlı", "düşük kalite"]
